# Stacking

For: NEIL HEINRICH BRAUN

The idea for this file is to train a lightGBM model (or other models) for stacking. The data files you will need to import is unfortunately not ready. For now, imagine a file in the following format

| ticker | quarter_year  | actual_log_rev| actual_car | log_revenue_prediction_1 | CAR_prediction_1 | log_revenue_prediction_2 | CAR_prediction_2 |
|--------|---------------|---------------|------------|--------------------------|------------------|--------------------------|------------------|
| BAC    | Q1 2001       |123            |0.4         | 123                      | 0.5              | 123                      | 0.5              |
| JPM    | Q1 2001       |123            |0.4         | 456                      | 0.8              | 456                      | 0.8              |
| WFC    | Q1 2001       |123            |0.4         | 789                      | 0.25             | 789                      | 0.25             |

There will be more columns where each column will be the prediction from each previous model for each y value.

You will need to train at least 2 models for stacking. 1 for each y value. Of course try other things outside of LightGBM as well but I suspect GBM to be the best. If you try other things outside of GBM, process the data accordingly to get a good range.Your output should be in the format:

| ticker | quarter_year  | log_revenue_prediction | CAR_prediction |
|--------|---------------|------------------------|----------------|
| BAC    | Q1 2001       | 123                    | 0.5            |
| JPM    | Q1 2001       | 456                    | 0.8            |
| WFC    | Q1 2001       | 789                    | 0.25           |

Enjoy!